# Reserve and configure FABRIC resources for "setting intermediate network"

## Set up your FABRIC environment

This assumes that you have already configured your FABRIC account and your Jupyter environment as described in [Hello, FABRIC](https://teaching-on-testbeds.github.io/blog/hello-fabric).

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
fablib.show_config()

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
Bastion Username,vinita_p_0000073925
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
Bastion Username,vinita_p_0000073925
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


In [2]:
!chmod 600 {fablib.get_bastion_key_filename()}
!chmod 600 {fablib.get_default_slice_private_key_file()}

## Create and submit a slice

In [3]:
cdn=True

In [ ]:
slice_name="intermediate-network-6"
slice = fablib.new_slice(name=slice_name)

This cell will try to identify a site that has sufficient resources for your experiment. But, you should still check in the [FABRIC Portal](https://portal.fabric-testbed.net/resources/all) to make sure it is not in maintenance or out of service - if so, you should run the cell again until you get a site that is ready to use.

In [ ]:
import random

sites=[]
total_routers=[]
routers_access_network=random.randint(3,6)
print("routers_access_network: ", routers_access_network)
if cdn:
    number_of_AS=random.randint(2,3)
    print("number_of_AS:", number_of_AS)
    routers_final_network=random.randint(3,4)  
    print("routers_final_network:", routers_final_network)
    for AS in range(number_of_AS):
        if AS==0:
            routers=[1]
            for i in range(routers_access_network-2):
                if i<2:
                    routers.append(1)
                else:
                    routers.append(random.randint(2,3))
            routers.append(1)
        elif AS==number_of_AS-1:
            routers=[1]
            for i in range(routers_final_network-2):
                routers.append(random.randint(2,3))
            routers.append(1)
        else:
            routers=[1]
            hops=random.randint(1,3)
            print("hops in "+str(AS)+" is "+str(hops))
            redundancy=True
            if redundancy:
                for i in range(0,hops-2):
                    routers.append(random.randint(1,3))
                routers.append(1)
            else:
                for i in range(hops-1):
                    routers.append(1)
        total_routers.append(routers)

else:
    number_of_AS=random.randint(3,5)
    print("number_of_AS:" ,number_of_AS)
    routers_final_network=random.randint(3,8)  
    print("routers_final_network:" ,routers_final_network)
    for AS in range(number_of_AS):
        if AS==0:
            routers=[1]
            for i in range(routers_access_network-2):
                if i<2:
                    routers.append(1)
                else:
                    routers.append(random.randint(1,3))
            routers.append(1)
        elif AS==number_of_AS-1:
            routers=[1]
            for i in range(routers_final_network-2):
                routers.append(random.randint(1,3))
            routers.append(1)
        else:
            routers=[1]
            hops=random.randint(1,3)
            print("hops in "+str(AS)+" is"+str(hops))
            redundancy=True
            if redundancy:
                for i in range(0,hops-2):
                    routers.append(random.randint(1,3))
                routers.append(1)
            else:
                for i in range(hops-1):
                    routers.append(1)
        total_routers.append(routers)

print(total_routers)

for r in total_routers:   
    exp_requires = {'core': 2*sum(r)*4, 'smart_nic_1': 2*sum(r)*1}
    while True:
        site_name = fablib.get_random_site()
        if ( (fablib.resources.get_core_available(site_name) > 1.2*exp_requires['core']) and
            (fablib.resources.get_component_available(site_name, 'SharedNIC-ConnectX-6') > 1.2**exp_requires['smart_nic_1']) ):
            break

    fablib.show_site(site_name)
    sites.append(site_name)

In [ ]:
file = open(slice_name+".txt", "w")
file.write(str(total_routers))
file.close()

In [ ]:
# this cell sets up the routers 
network_names_all = []

for i,router in enumerate(total_routers):
    network_names = []
    for j,r in enumerate(router):
        if r==1:
            network_names.append("as-"+str(i)+"-router-"+str(j)) 
        else:
            for k in range(r): 
                network_names.append("as-"+str(i)+"-router-"+str(j)+chr(k+97))
    network_names_all.append(network_names)

In [ ]:
print(network_names_all)

In [ ]:
for i,node in enumerate(network_names_all):
    site_name=sites[i]
    for n in node:
        print(n)
        slice.add_node(name=n, site=site_name, cores=8, ram=128, disk=25, image='default_ubuntu_22')

In [ ]:
# this cell sets up the network links 

for x,r in enumerate(total_routers):
    
    nets=[]
    skip1=0
    skip2=routers[0]

    for i in range(len(r)-1):

        j=i+1
        r1=r[i]
        r2=r[j]
        print(i,j)
        print(r1, r2)
        nodes=[]
        for k in range(r1):
            m=skip1+k
            print(m)
            nodes.append(network_names_all[x][m])
        for l in range(r2):
            nodes.append(network_names_all[x][skip2+l])
        nets.append({"name":"as-"+str(x)+"-net"+str(i)+"-"+str(j), "nodes": nodes})
        print("name:as-"+str(x)+"-net"+str(i)+"-"+str(j)+ ",nodes:", nodes)        
        skip1=skip1+r1
        skip2=skip2+r2
    print(nets)

    for n in nets:
        ifaces = [slice.get_node(node).add_component(model="NIC_Basic", name=n["name"]).get_interfaces()[0] for node in n['nodes'] ]
        slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)
                                                                   



When everything is ready, submit the slice and wait for it to get to "StableOK" state.  You can also see the state of your slice on the browser-based interface at https://portal.fabric-testbed.net/experiments#slices.

In case of an error, you can modify the slice name in the first cell of the "Create and submit a slice" section, to try again with a new slice (different name). Then, return to this cell and click Run > Run All Above Selected Cell. Finally, re-run the `slice.submit()` cell.

In [ ]:
slice.submit()

## Configure resources

In [ ]:
slice.get_state()

In [ ]:
slice = fablib.get_slice(name='intermediate-network-1')

In [ ]:
total_routers=[[1, 1, 1, 3, 2, 1], [1, 1, 1], [1, 3, 2, 1]]

In [ ]:
slice.wait_ssh(progress=True)

Bring up all of the network interfaces:

In [ ]:
for iface in slice.get_interfaces():
    iface.ip_link_up()

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv4Network
for x,routers in enumerate(total_routers):
    skip1=0
    if_name=[]
    for i in range(len(routers)-1):
        ip=1
        j=i+1
        r1=routers[i]
        r2=routers[j]
        print(i,j)
        print(r1, r2)

        for k in range(r1+r2):
            iname=slice.get_interface(network_names_all[x][k+skip1]+"-as-"+str(x)+"-net"+str(i)+"-"+str(j)+"-p1")
            print(iname.get_name())
            if_name.append(slice.get_interface(network_names_all[x][k+skip1]+"-as-"+str(x)+"-net"+str(i)+"-"+str(j)+"-p1"))
            iname.ip_addr_add(addr="10.10."+str(x+1)+str(i)+"."+str(ip), subnet=IPv4Network("10.10."+str(x+1)+str(i)+".0/24"))
            print("addr=10.10."+str(x+1)+str(i)+"."+str(ip)+", subnet=IPv4Network(10.10."+str(x+1)+str(i)+".0/24)")
            print(network_names_all[x][k+skip1]+"-as-"+str(x)+"-net"+str(i)+"-"+str(j)+"-p1") 
            ip=ip+1
        skip1=skip1+r1
        print(if_name)
    

#for iname in if_name:
#    iname.ip_addr_add(addr="10.10."+str(ip)+"."+str(ip), subnet=IPv4Network("10.10."+str(ip+1)+".0/24"))
#   print("addr=10.10."+str(ip)+"."+str(ip), "subnet=IPv4Network(10.10."+str(ip+1)+".0/24)")
#  ip=ip+1


In [ ]:
router_nodes=[]
for x,net in enumerate(network_names_all):
    nodes=[slice.get_node(name=n) for n in network_names_all[x]]
    router_nodes.append(nodes)
    

In [ ]:
print(len(router_nodes))

In [ ]:
print(router_nodes)

In [ ]:
for x,nodes in enumerate(router_nodes):
    for n in nodes:
        print(n.get_name())
        #n.execute("sudo sysctl -w net.ipv4.ip_forward=1")
        #n.execute("sudo apt-get update; sudo apt-get -y install net-tools", quiet=True)
        n.execute("wget -O - https://git.io/JYhs5 | bash", quiet=True)
    

In [ ]:
for x,nodes in enumerate(router_nodes):
    for n in nodes:
        print(n)
        n.execute("ifconfig -a")

In [ ]:
for x,nodes in enumerate(router_nodes):
    for n in nodes:
        n.execute("VTYSH_PAGER=more; sudo vtysh; sudo vtysh -E -c'configure terminal\nrouter ospf\nnetwork 10.10.0.0/16 area 0.0.0.0\n exit\n exit\n exit'  ")

In [ ]:
for x,nodes in enumerate(router_nodes):
    for n in nodes:
        n.execute("VTYSH_PAGER=more; sudo vtysh; sudo vtysh -E -c 'show ip route\nexit'")

In [ ]:
# slice.delete()

In [ ]:
for x,nodes in enumerate(router_nodes):
    for n in nodes:
        n.execute("wget -O - https://git.io/JYhs5 | bash")

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

# Set end date to 3 days from now
end_date = (datetime.now(timezone.utc) + timedelta(days=4)).strftime("%Y-%m-%d %H:%M:%S %z")
slice.renew(end_date)